In [ ]:
'''
@File        :alien_invasion.ipynb
@Description :
@Date        :2021/11/03 22:56:14
@Author      :Bo Sun
'''


import sys
from time import sleep

import pygame

from settings import Settings
from game_stats import GameStats
from ship import Ship
from bullet import Bullet
from alien import Alien

In [ ]:
class AlienInvasion:
    """
    manage game resources and behaviors
    """

    def __init__(self) -> None:
        pygame.init()
        
        self.settings = Settings()

        self.screen = pygame.display.set_mode(
            (self.settings.screen_width, 
            self.settings.screen_height))
        pygame.display.set_caption("Alien Invasion")

        self.stats = GameStats(self) # store statistics
        self.ship = Ship(self) # use self
        self.bullets = pygame.sprite.Group()
        self.aliens = pygame.sprite.Group()
        
        self._create_fleet()


    def run_game(self):
        """
        start the main loop
        """
        while True:
            self._check_events()

            if self.stats.game_active:
                self.ship.update()
                self._update_bullets()
                self._update_aliens()

            self._update_screen()

    def _update_screen(self):
        """
        update the image on the screen
        switch to the new screen
        """
        # fill the screen at each run
        self.screen.fill(self.settings.bg_color) 
        self.ship.blitme()

        for bullet in self.bullets.sprites():
            bullet.draw_bullet()

        self.aliens.draw(self.screen)

        # visualisation
        # Update the full display Surface to the screen
        pygame.display.flip()

    def _update_aliens(self):
        """"
        update the position of the alien
        """
        self._check_fleet_edges()
        self.aliens.update()

        # check the collision between the alien and the ship
        if pygame.sprite.spritecollideany(self.ship, self.aliens):
            self._ship_hit()

        # check whether there is a alien reaching the bottom
        self._check_alien_bottom()

    def _update_bullets(self):
        """
        update the bullet position 
        delete the vanishing bullet
        """
        # update the bullet position
        self.bullets.update()

        # delete the vanishing bullet
        for bullet in self.bullets.copy():
            if bullet.rect.bottom <= 0:
                self.bullets.remove(bullet)

        self._check_bullet_alien_collisions()

    def _check_events(self):
        """
        monitor & react to keyborad and mouse events
        """
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit() # quit pygame
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                self._check_keydown_events(event)
            elif event.type == pygame.KEYUP:
                self._check_keyup_events(event)

    def _check_keydown_events(self, event):
        """
        response to keydown
        """
        if event.key == pygame.K_RIGHT:
            # move the ship right-ward
            self.ship.moving_right = True
        elif event.key == pygame.K_LEFT:
            # move the ship left-ward
            self.ship.moving_left = True
        elif event.key == pygame.K_q:
            # quit
            pygame.quit() # quit pygame
            sys.exit()
        elif event.key == pygame.K_SPACE:
            self._fire_bullet()
    
    
    def _check_keyup_events(self, event):
        """
        response to keyup
        """
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = False
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = False



    def _fire_bullet(self):
        """
        creat a bullet and add it to the bullets group
        """
        if len(self.bullets) < self.settings.bullets_allowed:
            new_bullet = Bullet(self)
            self.bullets.add(new_bullet)




    
    def _check_bullet_alien_collisions(self):
        """
        react to the collistion between the bullet and the alien
        """
        # check the collision
        # if yes, del the bullet and the alien
        collisions = pygame.sprite.groupcollide(
            self.bullets, self.aliens, True, True)

        # del current bullet and creat a new fleet
        if not self.aliens:
            self.bullets.empty()
            self._create_fleet()

    def _create_fleet(self):
        """
        create the alien fleet
        """
        # make a alien.
        alien = Alien(self)
        alien_width, alien_height = alien.rect.size
        ship_height  = self.ship.rect.height
        
        available_space_x = self.settings.screen_width - (2*alien_width)
        number_aliens_x = available_space_x // (2*alien_width)
        available_space_y = self.settings.screen_height - (3*alien_height) - ship_height
        number_aliens_y = available_space_y // (2*alien_height)

        # create a row of aliens
        for alien_y_number in range(number_aliens_y):
            for alien_x_number in range(number_aliens_x):
                self._create_alien(alien_x_number, alien_y_number)

    def _create_alien(self, alien_x_number, alien_y_number):
        alien = Alien(self)
        alien_width, alien_height = alien.rect.size
        alien.x = alien_width + 2*alien_width*alien_x_number
        alien.rect.x = alien.x
        alien.y = alien_height + 2*alien_height*alien_y_number
        alien.rect.y = alien.y
        self.aliens.add(alien)


    def _check_fleet_edges(self):
        """"
        take measures when a alien reaches the edge
        """
        for alien in self.aliens.sprites():
            if alien.check_edge():
                self._change_fleet_direction()
                break

    def _change_fleet_direction(self):
        """
        move the fleet downward, and change the direction
        """
        for alien in self.aliens.sprites():
            alien.rect.y += self.settings.fleet_drop_speed
        self.settings.fleet_direction *= -1

    def _ship_hit(self):
        """
        react to the collision between the ship and the alien
        """
        if self.stats.ships_left > 0:
            # left ship -1
            self.stats.ships_left -= 1

            # empty left aliens and bullets
            self.aliens.empty()
            self.bullets.empty()

            # build a new fleet, and place the ship to the original position
            self._create_fleet()
            self.ship.center_ship()

            # pause
            sleep(0.5)

        else:
            self.stats.game_active = False

    def _check_alien_bottom(self):
        """
        check whether there is alien reaching the bottom
        """
        screen_rect = self.screen.get_rect()
        for alien in self.aliens.sprites():
            if alien.rect.bottom >= screen_rect.bottom:
                # behave like the ship is hit
                self._ship_hit()
                break









In [ ]:
# if __name__ == 'main':
#     ai = AlienInvasion()
#     ai.run_game()

ai = AlienInvasion()
ai.run_game()